# Assessment of number of measures

In [1]:
# required Python imports
import pandas as pd
import numpy as np
from types import SimpleNamespace

import ftir_funct as f
np.set_printoptions(suppress=True)

module FTIR v.2024.3.19 imported


## Generate database

In [11]:
# generate Euler angles each 7 degrees and store in the database
database = SimpleNamespace(euler=f.explore_Euler_space(step=7, lower_bounds=(0, 0, 0), upper_bounds=(89, 89, 179)))

In [12]:
dataset = pd.DataFrame(
    {
        "Euler1": database.euler[:, 0],
        "Euler2": database.euler[:, 1],
        "Euler3": database.euler[:, 2],
    }
)
dataset

,Euler1,Euler2,Euler3
0,0,0,0
1,0,0,7
2,0,0,14
3,0,0,21
4,0,0,28
...,...,...,...
4389,84,84,147
4390,84,84,154
4391,84,84,161
4392,84,84,168


In [4]:
# Transmission values for lambda 1987.29
Ta = 0.009882
Tb = 0.995815
Tc = 0.596951

# Generate a mesh of values defining the reference transmissión envelope
polar, azimuths = f.regular_S2_grid(n_squared=500)
T = f.Tvalues(trans=(Ta, Tb, Tc), azimuth=azimuths, polar=polar)
x, y, z = f.sph2cart(T, azimuths, polar)


In [5]:
np.arange(0, 360, 90.01)

array([  0.  ,  90.01, 180.02, 270.03])

In [6]:
steps = (90.01, 45.0, 22.5, 15.0)  # in degrees
columns = ["4samples", "8samples", "16samples", "24samples"]

for i, step in enumerate(steps):
    # create database
    angles = np.arange(0, 360, step)
    T_vals = []
    azi_vals = []

    for euler in database.euler:
        # rotate
        x2, y2, z2 = f.rotate(coordinates=(x, y, z), euler_ang=euler)

        # extract XY intersection
        xy_vectors = f.extract_XY_section_fast2(x2, y2, z2)

        # get the indexes of specific angles
        indexes = f.find_nearest(xy_vectors["angles"], angles)

        # append values
        T_vals.append(xy_vectors.loc[indexes, ["T"]].T.values.tolist()[0])
        azi_vals.append(xy_vectors.loc[indexes, ["angles"]].T.values.tolist()[0])

    database.T_values = np.array(T_vals)
    database.azimuths = np.array(azi_vals)

    # estimate orientations
    disorientations = np.empty(database.euler.shape[0])

    for index, orientation in enumerate(database.euler):
        measures = np.column_stack(
            (
                database.T_values[index],
                database.azimuths[index],
                np.full_like(database.azimuths[index], 90),
            )
        )

        estimate = f.find_orientation(measurements=measures, params=(Ta, Tb, Tc))
        disorientations[index] = f.calc_disorientation(orientation, estimate.x)

    
    # add column
    dataset[columns[i]] = disorientations

    # clean database
    del database.T_values
    del database.azimuths


In [7]:
dataset

,Euler1,Euler2,Euler3,4samples,8samples,16samples,24samples
0,0,0,0,0.000,0.000,0.000,0.000
1,0,0,75,0.001,0.000,0.286,0.000
2,0,0,150,0.000,0.000,0.361,0.298
3,0,75,0,0.003,0.002,0.000,0.222
4,0,75,75,5.688,0.251,0.263,0.222
5,0,75,150,9.737,0.027,0.300,0.212
6,75,0,0,0.000,0.000,0.000,0.232
7,75,0,75,0.000,0.442,0.286,0.227
8,75,0,150,2.833,0.310,0.398,0.226
9,75,75,0,3.234,0.011,0.003,0.006


## Generate dataframe and save data

In [8]:
from datetime import date    
today = date.today().isoformat()

In [9]:
dataset.to_csv(today + '_step7_lamb1987_measures.csv', index=False)

In [10]:
# quick statistical overview
dataset[['4samples', '8samples', '16samples', '24samples']].describe()

,4samples,8samples,16samples,24samples
count,12.000000,12.000000,12.000000,12.000000
mean,3.359250,0.087833,0.159083,0.138000
std,5.390014,0.154568,0.167299,0.120723
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.002250,0.005750
50%,0.638000,0.005500,0.135000,0.217000
75%,3.847500,0.083000,0.289500,0.226250
max,17.542000,0.442000,0.398000,0.298000
